In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   try:
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     print(e)

In [4]:
train_dir = "./data/train"
test_dir = "./data/test"

class earlyCancel(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('loss')<0.3):
            print("\nLoss is low so cancelling training!")
            self.model.stop_training = True

earlycancel = earlyCancel()

In [5]:
train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     rotation_range=40, # range from 0 to 180 degrees; random
#     width_shift_range=0.2, # shift object from center (as proportion of image size); how does it know object??
#     height_shift_range=0.2, # shifts object from center(as proportion of image size); how does it know object?? 
#     shear_range=0.2, # by portion of image; 0.2 will shear up to 20% of image
#     zoom_range=0.2, # relative portion too. 
#     horizontal_flip=True,
#     vertical_flip=True,
#     fill_mode='nearest',
    validation_split=0.05,
    )


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224), # this is useful for loading in data that isn't this size; will resize as it's loaded in
    batch_size=32, # whole science for selecting this; beyond scope of this class
    class_mode='binary',
    subset='training'
    )

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224,224), # this is useful for loading in data that isn't this size; will resize as it's loaded in
    batch_size=128, # whole science for selecting this; beyond scope of this class
    class_mode='binary',
    subset='validation'
    )

Found 2506 images belonging to 2 classes.
Found 131 images belonging to 2 classes.


In [6]:
inputs = tf.keras.Input(shape=(224, 224, 3, ))

rscl = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255)(inputs)

c2d1 = tf.keras.layers.Conv2D(8, (3,3), activation='swish', padding='Same')(rscl)

avp1 = tf.keras.layers.AveragePooling2D(2, 2)(c2d1)
c2d2a = tf.keras.layers.Conv2D(16, (3,3), activation='swish', padding='Same')(avp1)
avp2 = tf.keras.layers.AveragePooling2D(2, 2)(c2d2a)
c2d3a = tf.keras.layers.Conv2D(32, (3,3), activation='swish', padding='Same')(avp2)
avp3 = tf.keras.layers.AveragePooling2D(2, 2)(c2d3a)
c2d4a = tf.keras.layers.Conv2D(64, (3,3), activation='swish', padding='Same')(avp3)
avp4 = tf.keras.layers.AveragePooling2D(2, 2)(c2d4a)

mxp1 = tf.keras.layers.MaxPool2D(2, 2)(c2d1)
c2d2m = tf.keras.layers.Conv2D(16, (3,3), activation='swish', padding='Same')(mxp1)
mxp2 = tf.keras.layers.MaxPool2D(2, 2)(c2d2m)
c2d3m = tf.keras.layers.Conv2D(32, (3,3), activation='swish', padding='Same')(mxp2)
mxp3 = tf.keras.layers.MaxPool2D(2, 2)(c2d3m)
c2d4m = tf.keras.layers.Conv2D(64, (3,3), activation='swish', padding='Same')(mxp3)
mxp4 = tf.keras.layers.MaxPool2D(2, 2)(c2d4m)

ctc = tf.keras.layers.Concatenate()([avp4, mxp4])

flt = tf.keras.layers.Flatten()(ctc)

dns = tf.keras.layers.Dense(512, activation='swish')(flt)

drp = tf.keras.layers.Dropout(0.2)(dns)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(drp)

In [7]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [8]:
model.compile(loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(amsgrad=True),
    metrics=['acc'])

In [9]:
history = model.fit(
    train_generator, # generator that we set up earlier; streams images from directory
    epochs=25, 
    validation_data=val_generator,
    callbacks=[earlycancel])

Epoch 1/25
79/79 [==============================] - 12s 103ms/step - loss: 0.8607 - acc: 0.5851 - val_loss: 0.6711 - val_acc: 0.6260
Epoch 2/25
79/79 [==============================] - 7s 86ms/step - loss: 0.5497 - acc: 0.7003 - val_loss: 0.4862 - val_acc: 0.7176
Epoch 3/25
79/79 [==============================] - 7s 88ms/step - loss: 0.4251 - acc: 0.7959 - val_loss: 0.4843 - val_acc: 0.7481
Epoch 4/25
79/79 [==============================] - 7s 85ms/step - loss: 0.3870 - acc: 0.8201 - val_loss: 0.3533 - val_acc: 0.8244
Epoch 5/25
79/79 [==============================] - 7s 88ms/step - loss: 0.3670 - acc: 0.8167 - val_loss: 0.5359 - val_acc: 0.7634
Epoch 6/25
79/79 [==============================] - 7s 84ms/step - loss: 0.3929 - acc: 0.8139 - val_loss: 0.4866 - val_acc: 0.7405
Epoch 7/25
79/79 [==============================] - 7s 86ms/step - loss: 0.3614 - acc: 0.8193 - val_loss: 0.3456 - val_acc: 0.8321
Epoch 8/25
79/79 [==============================] - 7s 87ms/step - loss: 0.3327 -

In [10]:
model.save('cancer.keras')

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 244, 244, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 244, 244, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 244, 244, 8)  224         rescaling[0][0]                  
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 122, 122, 8)  0           conv2d[0][0]                     
______________________________________________________________________________________________

In [12]:
test_datagen = ImageDataGenerator(
#     rescale=1./255
)

In [13]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(244,244),
    batch_size=32,
    class_mode='binary')

Found 660 images belonging to 2 classes.


In [14]:
model.evaluate(test_generator)

21/21 [==============================] - 2s 83ms/step - loss: 0.3654 - acc: 0.8258


[0.36543479561805725, 0.8257575631141663]